In [2]:
import os
from pathlib import Path
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
import time

In [6]:
# Configuración de Azure Form Recognizer
endpoint = "https://docint-demo-adium.cognitiveservices.azure.com/"
api_key = "6VycicjSZOGX5jMomqRIXRCFgoiSs7v1f6Ua1uL9xXj0BaOqyUm9JQQJ99BFACYeBjFXJ3w3AAALACOGyOXC"
client = DocumentAnalysisClient(endpoint, AzureKeyCredential(api_key))

def extraer_texto_pdf(ruta_pdf, client):
    #Usamos prebuilt-read para extraer texto de documentos PDF, es una opción usar prebuilt-layout para tablas
    try:
        with open(ruta_pdf, "rb") as f:
            poller = client.begin_analyze_document("prebuilt-read", f)
            result = poller.result()
            
        texto_extraido = "\n".join([line.content for page in result.pages for line in page.lines])
        return texto_extraido
    except Exception as e:
        print(f"Error procesando {ruta_pdf}: {str(e)}")
        return None

In [ ]:
def procesar_carpeta_casos(ruta_base, carpeta_salida="extract"):
    
    ruta_base = Path(ruta_base)
    
    #Crear carpeta de salida si no existe
    carpeta_salida_path = ruta_base / carpeta_salida
    carpeta_salida_path.mkdir(exist_ok=True)
    
    total_pdfs = 0
    procesados_exitosamente = 0
    errores = 0
    
    for caso_carpeta in ruta_base.iterdir():
        if caso_carpeta.is_dir() and caso_carpeta.name.startswith("Caso_"):
            print(f"\nProcesando {caso_carpeta.name}...")
            
            #Crear subcarpeta
            subcarpeta_salida = carpeta_salida_path / caso_carpeta.name
            subcarpeta_salida.mkdir(exist_ok=True)
            
            pdfs_encontrados = list(caso_carpeta.rglob("*.pdf"))
            
            if not pdfs_encontrados:
                print(f"No se encontraron PDFs en {caso_carpeta.name}")
                continue
            
            for pdf_path in pdfs_encontrados:
                total_pdfs += 1
                print(f"Procesando: {pdf_path.name}")
                
                texto = extraer_texto_pdf(pdf_path, client)
                
                if texto is not None:
                    nombre_salida = pdf_path.stem + "_extraido.txt"
                    ruta_salida = subcarpeta_salida / nombre_salida
                    
                    #Texto extraído
                    try:
                        with open(ruta_salida, "w", encoding="utf-8") as out:
                            out.write(f"=== TEXTO EXTRAÍDO DE: {pdf_path.name} ===\n")
                            out.write(f"Ruta original: {pdf_path}\n")
                            out.write(f"Fecha de procesamiento: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                            out.write("=" * 50 + "\n\n")
                            out.write(texto)
                        
                        print(f"Guardado en: {ruta_salida}")
                        procesados_exitosamente += 1
                        
                    except Exception as e:
                        print(f"Error guardando archivo: {str(e)}")
                        errores += 1
                else:
                    print(f"Error extrayendo texto")
                    errores += 1
                
                #ByPass:
                time.sleep(0.5)

In [9]:
def listar_estructura_carpetas(ruta_base):

    ruta_base = Path(ruta_base)
    print(f"Estructura de carpetas en: {ruta_base}")
    print("=" * 50)
    
    for caso_carpeta in sorted(ruta_base.iterdir()):
        if caso_carpeta.is_dir() and caso_carpeta.name.startswith("Caso_"):
            print(f"\n{caso_carpeta.name}")
            
            # Buscar PDFs en subcarpetas
            pdfs = list(caso_carpeta.rglob("*.pdf"))
            
            if pdfs:
                for pdf in sorted(pdfs):
                    # Mostrar ruta relativa desde la carpeta del caso
                    ruta_relativa = pdf.relative_to(caso_carpeta)
                    print(f"{ruta_relativa}")
            else:
                print("  ⚠️  No se encontraron PDFs")

In [12]:
ruta_base = "/Users/rayespinoza/AI_Projects/cordoba_classificator/data/casos"
print("Extraer información de los PDFs")
print("=" * 50)
listar_estructura_carpetas(ruta_base)

Extraer información de los PDFs
Estructura de carpetas en: /Users/rayespinoza/AI_Projects/cordoba_classificator/data/casos

Caso_1
1-Corrida de Vista/caso1_corrida_de_vista-SD.pdf
2-Aporte del contribuyente/caso1_aporte_1-SD.pdf
2-Aporte del contribuyente/caso1_aporte_2-SD.pdf
2-Aporte del contribuyente/caso1_aporte_3-SD.pdf
2-Aporte del contribuyente/caso1_aporte_4-SD.pdf
2-Aporte del contribuyente/caso1_aporte_5-SD.pdf
3-Resolución Determinativa/caso1_resolucion_determinativa-SD.pdf

Caso_2
1- Corrida de Vista/caso2_corrida_de_vista-SD.pdf
2-Aporte del contribuyente/caso2_aporte_1-SD.pdf
2-Aporte del contribuyente/caso2_aporte_2-SD.pdf
2-Aporte del contribuyente/caso2_aporte_3-SD.pdf
2-Aporte del contribuyente/caso2_aporte_4-SD.pdf
2-Aporte del contribuyente/caso2_aporte_5-SD.pdf
2-Aporte del contribuyente/caso2_aporte_6-SD.pdf
3- Resolución/caso2_resolucion_determinativa-SD.pdf

Caso_3
1- Corrida de Vista/caso3_corrida_de_vista-SD.pdf
2-Aporte del contribuyente/caso3_descargo_1-SD

In [ ]:
ruta_base = "/Users/rayespinoza/AI_Projects/cordoba_classificator/data/casos"
print("Extraer información de los PDFs")
print("=" * 50)
listar_estructura_carpetas(ruta_base)
resultados = procesar_carpeta_casos(ruta_base)
print(f"Revisa la carpeta '{resultados['carpeta_salida']}' para ver los archivos extraídos.")